In [4]:
import torch
class NeutralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(
            # first layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            # second layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        return self.layers(x)

if __name__ == "__main__":
    model = NeutralNetwork(5, 1)
    x = torch.randn(1, 5)
    print(model(x))
    print(model)

tensor([[0.0149]], grad_fn=<AddmmBackward0>)
NeutralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=5, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)
